In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 7s 5ms/step - loss: 0.3651 - sparse_categorical_accuracy: 0.8991 - val_loss: 0.2006 - val_sparse_categorical_accuracy: 0.9434
Epoch 2/6
469/469 [==============================] - 2s 5ms/step - loss: 0.1708 - sparse_categorical_accuracy: 0.9525 - val_loss: 0.1502 - val_sparse_categorical_accuracy: 0.9560
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1221 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.1227 - val_sparse_categorical_accuracy: 0.9633
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0954 - sparse_categorical_accuracy: 0.9728 - val_loss: 0.1001 - val_sparse_categorical_accuracy: 0.9704
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0771 - sparse_categorical_accuracy: 0.9776 - val_loss: 0.0858 - val_sparse_categorical_accuracy: 0.9741
Epoch 6/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0629 - sparse_categorical_accu

In [ ]:
a = model.predict(ds_test)
a


 1/79 [..............................] - ETA: 1s

79/79 [==============================] - 0s 2ms/step


array([[-4.4943962e+00, -7.0729389e+00,  1.1346147e+01, ...,
        -7.0380011e+00,  2.9859281e+00, -6.2184367e+00],
       [ 1.7343691e+01, -1.1755932e+01, -9.0459166e+00, ...,
        -1.2060841e+00, -1.8235246e+00, -8.4472618e+00],
       [-7.8066421e+00, -1.2072872e+01, -3.9114134e+00, ...,
         7.5327396e-01, -5.2304864e-01,  3.2495322e+00],
       ...,
       [-4.7226081e+00, -6.5016046e+00,  2.1722791e+00, ...,
        -1.0637643e+01,  9.7718000e+00, -4.8748159e+00],
       [ 1.4307515e+01, -1.2362142e+01,  2.8892257e+00, ...,
        -4.8695226e+00, -6.9461112e+00, -8.8793188e-03],
       [-2.0663071e+00, -5.6922612e+00, -3.8091278e+00, ...,
        -7.7421746e+00,  7.5815302e-01, -3.0677373e+00]], dtype=float32)

In [ ]:
print("Evaluate model on test data")
results = model.evaluate(ds_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate model on test data
 1/79 [..............................] - ETA: 1s - loss: 0.0470 - sparse_categorical_accuracy: 0.9766

79/79 [==============================] - 0s 5ms/step - loss: 0.0860 - sparse_categorical_accuracy: 0.9734
test loss, test acc: [0.08603741973638535, 0.9733999967575073]
